# Add upstream, downstream and basin information to the dataframe

* Purpose of script: add contextual data to the datafram. 
* Author: Rutger Hofste
* Kernel used: python35
* Date created: 20170915

In [1]:
import time
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
print(dateString,timeString)

Y2017M09D28 UTC 15:37


In [2]:
S3_INPUT_PATH_EE  = "s3://wri-projects/Aqueduct30/processData/Y2017M09D14_RH_merge_EE_results_V01/output/"
S3_OUTPUT_PATH = "s3://wri-projects/Aqueduct30/processData/Y2017M09D15_RH_Add_Basin_Data_V01/output/"

S3_INPUT_PATH_HYDROBASINS = "s3://wri-projects/Aqueduct30/processData/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/"

EC2_INPUT_PATH = "/volumes/data/Y2017M09D15_RH_Add_Basin_Data_V01/input"
EC2_OUTPUT_PATH = "/volumes/data/Y2017M09D15_RH_Add_Basin_Data_V01/output"

INPUT_FILENAME_EE =  "mergedZonalStatsEE_V12.pkl"
INPUT_FILENAME_HYDROBASINS =  "hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.pkl"

OUTPUT_FILENAME = "Y2017M09D15_RH_Add_Basin_Data_V01"


Note: There are two polygons with the same PFAF_ID (353020). This is caused by the fact that both poygons would otherwise cross the 180 degree meridian

In [3]:
!rm -r {EC2_INPUT_PATH} 
!rm -r {EC2_OUTPUT_PATH} 

In [4]:
!mkdir -p {EC2_INPUT_PATH} 
!mkdir -p {EC2_OUTPUT_PATH} 

In [5]:
!aws s3 cp {S3_INPUT_PATH_EE} {EC2_INPUT_PATH} --recursive

download: s3://wri-projects/Aqueduct30/processData/Y2017M09D14_RH_merge_EE_results_V01/output/mergedZonalStatsEE_V12.pkl to ../../../../data/Y2017M09D15_RH_Add_Basin_Data_V01/input/mergedZonalStatsEE_V12.pkl
download: s3://wri-projects/Aqueduct30/processData/Y2017M09D14_RH_merge_EE_results_V01/output/mergedZonalStatsEE_V12.csv to ../../../../data/Y2017M09D15_RH_Add_Basin_Data_V01/input/mergedZonalStatsEE_V12.csv


In [6]:
!aws s3 cp {S3_INPUT_PATH_HYDROBASINS} {EC2_INPUT_PATH} --recursive --exclude "*" --include "*.pkl"

download: s3://wri-projects/Aqueduct30/processData/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.pkl to ../../../../data/Y2017M09D15_RH_Add_Basin_Data_V01/input/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.pkl


In [7]:
import os
import pandas as pd
import multiprocessing as mp
import pickle
import numpy as np
import itertools
import logging
import pprint
import ast

In [8]:
inputLocationEE = os.path.join(EC2_INPUT_PATH,INPUT_FILENAME_EE)
inputLocationHydroBasins = os.path.join(EC2_INPUT_PATH,INPUT_FILENAME_HYDROBASINS)

In [9]:
df_ee = pd.read_pickle(inputLocationEE)

In [10]:
df_ee.index.names = ['PFAF_ID']

In [11]:
df_HydroBasins = pd.read_pickle(inputLocationHydroBasins)


In [12]:
df_complete = df_HydroBasins.merge(df_ee,how="left",left_index=True, right_index=True)

Note: There are two polygons with the same PFAF_ID (353020). This is caused by the fact that both poygons would otherwise cross the 180 degree meridian. 

In [13]:
df_complete = df_complete.drop_duplicates(subset='PFAF_ID', keep='first')

## Functions

In [14]:
def calculateTotalDemand(useType,temporalResolution,year,month):
    # This function will add Dom Ind IrrLinear and Livestock of all basins in the input list
    
    if temporalResolution == "year":
        keyTotal = "local_sum_volumem3_Tot%s_%s_Y%0.4d" %(useType,temporalResolution,year)
    else:
        keyTotal = "local_sum_volumem3_Tot%s_%s_Y%0.4dM%0.2d" %(useType,temporalResolution,year,month)
    
    # Create Column with zeros
    dfDemand[keyTotal] = 0
    for demandType in demandTypes:
        if demandType == "IrrLinear" and temporalResolution == "year":
            key = "total_volume_%s%s_%sY%0.4d" %(demandType,useType,temporalResolution,year)
        else:
            key = "total_volume_%s%s_%sY%0.4dM%0.2d" %(demandType,useType,temporalResolution,year,month)
        dfDemand[keyTotal] = dfDemand[keyTotal] + df_complete[key]
    return dfDemand   


def addUpstream2(listje):
    df_full_temp = df_complete.copy()
    df_part_temp = df_full_temp[df_full_temp.index.isin(listje)]
    df_part_temp2 = df_part_temp.copy()
    df_out = df_part_temp2.copy()
    
    i = 0
    for index, row in df_part_temp2.iterrows():
        i += 1
        print("i: ",i  ," index: ", index)
        try:
            upstreamCatchments = df_part_temp2.loc[index, "Upstream_PFAF_IDs"]
            upstreamCatchments = ast.literal_eval(upstreamCatchments)
            df_upstream = df_full_temp[df_full_temp.index.isin(upstreamCatchments)]
            # selecting columns based on regular expression
            df_upstream = df_upstream.filter(regex=("total*"))
            df_upstream = df_upstream.add_prefix("upstream_")
            sumSeries = df_upstream.sum(0)
            for key, value in sumSeries.iteritems():
                df_out.loc[index, key] = value
            df_out.loc[index, "errorCode"] = 0
        except:
            print("error")
            df_out.loc[index, "errorCode"] = 1
            pass
    
    return df_out
    
    
def addDownstream2(listje):
    df_full_temp = df_complete.copy()
    df_part_temp = df_full_temp[df_full_temp.index.isin(listje)]
    df_part_temp2 = df_part_temp.copy()
    df_out = df_part_temp2.copy()
    
    i = 0
    for index, row in df_part_temp2.iterrows():
        i += 1
        print("i: ",i  ," index: ", index)
        try:
            upstreamCatchments = df_part_temp2.loc[index, "Downstream_PFAF_IDs"]
            upstreamCatchments = ast.literal_eval(upstreamCatchments)
            df_upstream = df_full_temp[df_full_temp.index.isin(upstreamCatchments)]
            # selecting columns based on regular expression
            df_upstream = df_upstream.filter(regex=("total*"))
            df_upstream = df_upstream.add_prefix("downstream_")
            sumSeries = df_upstream.sum(0)
            for key, value in sumSeries.iteritems():
                df_out.loc[index, key] = value
            df_out.loc[index, "errorCode"] = 0
        except:
            print("error")
            df_out.loc[index, "errorCode"] = 1
            pass
    
    return df_out


def addBasin2(listje):
    df_full_temp = df_complete.copy()
    df_part_temp = df_full_temp[df_full_temp.index.isin(listje)]
    df_part_temp2 = df_part_temp.copy()
    df_out = df_part_temp2.copy()
    
    i = 0
    for index, row in df_part_temp2.iterrows():
        i += 1
        print("i: ",i  ," index: ", index)
        try:
            upstreamCatchments = df_part_temp2.loc[index, "Basin_PFAF_IDs"]
            upstreamCatchments = ast.literal_eval(upstreamCatchments)
            df_upstream = df_full_temp[df_full_temp.index.isin(upstreamCatchments)]
            # selecting columns based on regular expression
            df_upstream = df_upstream.filter(regex=("total*"))
            df_upstream = df_upstream.add_prefix("basin_")
            sumSeries = df_upstream.sum(0)
            for key, value in sumSeries.iteritems():
                df_out.loc[index, key] = value
            df_out.loc[index, "errorCode"] = 0
        except:
            print("error")
            df_out.loc[index, "errorCode"] = 1
            pass
    
    return df_out

## Script

In [15]:
demandTypes = ["PDom","PInd","IrrLinear","PLiv"]
useTypes = ["WW","WN"]
temporalResolutions = ["year","month"]
years = [2014]

In [16]:
dfDemand = pd.DataFrame(index=df_complete.index)
for temporalResolution in temporalResolutions:
    for useType in useTypes:
        for year in years:
            if temporalResolution == "year":
                month = 12
                print(useType,temporalResolution,year,month)
                dfDemand = calculateTotalDemand(useType,temporalResolution,year,month)
            else:
                for month in range(1,13):
                    print(useType,temporalResolution,year,month)
                    dfDemand = calculateTotalDemand(useType,temporalResolution,year,month)          

WW year 2014 12
WN year 2014 12
WW month 2014 1
WW month 2014 2
WW month 2014 3
WW month 2014 4
WW month 2014 5
WW month 2014 6
WW month 2014 7
WW month 2014 8
WW month 2014 9
WW month 2014 10
WW month 2014 11
WW month 2014 12
WN month 2014 1
WN month 2014 2
WN month 2014 3
WN month 2014 4
WN month 2014 5
WN month 2014 6
WN month 2014 7
WN month 2014 8
WN month 2014 9
WN month 2014 10
WN month 2014 11
WN month 2014 12


In [17]:
dfDemand.head()

,local_sum_volumem3_TotWW_year_Y2014,local_sum_volumem3_TotWN_year_Y2014,local_sum_volumem3_TotWW_month_Y2014M01,local_sum_volumem3_TotWW_month_Y2014M02,local_sum_volumem3_TotWW_month_Y2014M03,local_sum_volumem3_TotWW_month_Y2014M04,local_sum_volumem3_TotWW_month_Y2014M05,local_sum_volumem3_TotWW_month_Y2014M06,local_sum_volumem3_TotWW_month_Y2014M07,local_sum_volumem3_TotWW_month_Y2014M08,...,local_sum_volumem3_TotWN_month_Y2014M03,local_sum_volumem3_TotWN_month_Y2014M04,local_sum_volumem3_TotWN_month_Y2014M05,local_sum_volumem3_TotWN_month_Y2014M06,local_sum_volumem3_TotWN_month_Y2014M07,local_sum_volumem3_TotWN_month_Y2014M08,local_sum_volumem3_TotWN_month_Y2014M09,local_sum_volumem3_TotWN_month_Y2014M10,local_sum_volumem3_TotWN_month_Y2014M11,local_sum_volumem3_TotWN_month_Y2014M12
PFAF_ID,,,,,,,,,,,,,,,,,,,,,
111011,6.257741e+07,2.915234e+07,5.071952e+06,5.039243e+06,5.096070e+06,5.268799e+06,5.233163e+06,5.282180e+06,5.382082e+06,5.472195e+06,...,2.372698e+06,2.448676e+06,2.443976e+06,2.468962e+06,2.509266e+06,2.543447e+06,2.498844e+06,2.432985e+06,2.380002e+06,2.353863e+06
111012,8.204290e+06,3.388488e+06,6.827566e+05,6.829166e+05,6.831846e+05,6.836583e+05,6.840569e+05,6.843250e+05,6.844042e+05,6.844215e+05,...,2.818678e+05,2.823415e+05,2.827401e+05,2.830082e+05,2.830874e+05,2.831047e+05,2.829387e+05,2.826311e+05,2.820958e+05,2.816334e+05
111013,7.569537e+06,3.111568e+06,6.301277e+05,6.302355e+05,6.304261e+05,6.307669e+05,6.310496e+05,6.312474e+05,6.313145e+05,6.313229e+05,...,2.589287e+05,2.592695e+05,2.595522e+05,2.597500e+05,2.598171e+05,2.598255e+05,2.597075e+05,2.594838e+05,2.591036e+05,2.587620e+05
111014,1.046668e+06,5.024309e+05,8.651350e+04,8.664306e+04,8.685163e+04,8.721648e+04,8.751861e+04,8.770851e+04,8.775453e+04,8.777285e+04,...,4.149858e+04,4.186342e+04,4.216556e+04,4.235545e+04,4.240148e+04,4.241980e+04,4.227204e+04,4.204755e+04,4.165468e+04,4.130191e+04
111015,1.118431e+08,4.668647e+07,9.301079e+06,9.303778e+06,9.309980e+06,9.319181e+06,9.327492e+06,9.333573e+06,9.335649e+06,9.335846e+06,...,3.881478e+06,3.889610e+06,3.896934e+06,3.902287e+06,3.904112e+06,3.904281e+06,3.900583e+06,3.894476e+06,3.886032e+06,3.877061e+06


In [18]:
df_complete = df_complete.merge(dfDemand,how="left",left_index=True,right_index=True)

In [19]:
mp.cpu_count()

8

In [20]:
indices_full = df_complete.index.values
indices_split = np.array_split(indices_full, mp.cpu_count())

In [21]:
indices_split[1].shape

(2050,)

In [22]:
mp.log_to_stderr()

In [23]:
logger = mp.get_logger()
logger.setLevel(logging.INFO)

In [24]:
pool = mp.Pool(mp.cpu_count())

[INFO/ForkPoolWorker-1] child process calling self.run()
[INFO/ForkPoolWorker-2] child process calling self.run()
[INFO/ForkPoolWorker-3] child process calling self.run()
[INFO/ForkPoolWorker-4] child process calling self.run()
[INFO/ForkPoolWorker-5] child process calling self.run()
[INFO/ForkPoolWorker-6] child process calling self.run()
[INFO/ForkPoolWorker-7] child process calling self.run()
[INFO/ForkPoolWorker-8] child process calling self.run()


i:  1  index:  231609
i:  1  index:  146660
i:  1  index:  111011
i:  1  index:  312977
i:  1  index:  435045
i:  1  index:  634207
i:  1  index:  561109
i:  1  index:  742977
i:  2  index:  435046
i:  2  index:  634208
i:  2  index:  231700
i:  2  index:  146670
i:  2  index:  111012
i:  2  index:  561200
i:  2  index:  742978
i:  2  index:  312978
i:  3  index:  435047
i:  3  index:  634209
i:  3  index:  146680
i:  3  index:  231801
i:  3  index:  561210
i:  3  index:  111013
i:  3  index:  742979
i:  3  index:  312979
i:  4  index:  634301
i:  4  index:  435048
i:  4  index:  146690
i:  4  index:  231802
i:  4  index:  561220
i:  4  index:  742981
i:  4  index:  111014
i:  4  index:  312980
i:  5  index:  634302
i:  5  index:  146700
i:  5  index:  231803
i:  5  index:  435049
i:  5  index:  561230
i:  5  index:  111015
i:  5  index:  312990
i:  5  index:  742982
i:  6  index:  634303
i:  6  index:  146801
i:  6  index:  435050
i:  6  index:  231804
i:  6  index:  561240
i:  6  ind

i:  45  index:  751404
i:  46  index:  436626
i:  46  index:  634707
i:  46  index:  232280
i:  46  index:  112014
i:  46  index:  313630
i:  46  index:  151134
i:  47  index:  561507
i:  47  index:  436627
i:  47  index:  634708
i:  46  index:  751405
i:  47  index:  232290
i:  47  index:  112015
i:  47  index:  313640
i:  47  index:  151135
i:  48  index:  561508
i:  48  index:  436628
i:  48  index:  634709
i:  48  index:  112016
i:  47  index:  751406
i:  48  index:  232301
i:  48  index:  151136
i:  49  index:  561509
i:  48  index:  313650
i:  49  index:  436629
i:  49  index:  634801
i:  49  index:  112017
i:  49  index:  232302
i:  48  index:  751407
i:  49  index:  151137
i:  50  index:  561601
i:  49  index:  313660
i:  50  index:  436631
i:  50  index:  634802
i:  50  index:  112018
i:  50  index:  232303
i:  50  index:  151138
i:  49  index:  751408
i:  51  index:  561602
i:  50  index:  313670
i:  51  index:  436632
i:  51  index:  634803
i:  51  index:  112019
i:  51  ind

i:  92  index:  635201
i:  89  index:  752101
i:  92  index:  561960
i:  89  index:  151205
i:  90  index:  436690
i:  91  index:  112803
i:  91  index:  322212
i:  91  index:  232708
i:  93  index:  635202
i:  93  index:  561970
i:  90  index:  151206
i:  91  index:  436691
i:  90  index:  752102
i:  92  index:  112804
i:  92  index:  322213
i:  92  index:  232709
i:  94  index:  635203
i:  94  index:  561981
i:  91  index:  151210
i:  92  index:  436692
i:  93  index:  112805
i:  93  index:  322214
i:  91  index:  752103
i:  93  index:  232810
i:  95  index:  635204
i:  95  index:  561982
i:  92  index:  151220
i:  93  index:  436693
i:  94  index:  112806
i:  94  index:  322215
i:  94  index:  232820
i:  92  index:  752104
i:  96  index:  635205
i:  96  index:  561983
i:  93  index:  151221
i:  94  index:  436694
i:  95  index:  112807
i:  95  index:  322216
i:  95  index:  232830
i:  93  index:  752105
i:  97  index:  635206
i:  97  index:  561984
i:  95  index:  436695
i:  94  ind

i:  134  index:  234001
i:  131  index:  752163
i:  134  index:  114289
i:  133  index:  322330
i:  132  index:  151603
i:  136  index:  635609
i:  136  index:  562387
i:  135  index:  234002
i:  134  index:  436900
i:  132  index:  752164
i:  135  index:  114290
i:  134  index:  322340
i:  133  index:  151604
i:  137  index:  635700
i:  137  index:  562388
i:  135  index:  437001
i:  136  index:  234003
i:  133  index:  752165
i:  135  index:  322350
i:  136  index:  114300
i:  134  index:  151605
i:  138  index:  635801
i:  138  index:  562389
i:  136  index:  437002
i:  134  index:  752166
i:  137  index:  234004
i:  136  index:  322360
i:  137  index:  114410
i:  135  index:  151606
i:  139  index:  635802
i:  139  index:  562390
i:  137  index:  437003
i:  135  index:  752167
i:  138  index:  234005
i:  137  index:  322370
i:  138  index:  114411
i:  136  index:  151607
i:  140  index:  635803
i:  140  index:  562410
i:  138  index:  437004
i:  136  index:  752168
i:  139  index: 

...

i:  1902  index:  226067
i:  1896  index:  631065
i:  1904  index:  742683
i:  1903  index:  434430
i:  1903  index:  912705
i:  1898  index:  144502
i:  1899  index:  312549
i:  1900  index:  531101
i:  1903  index:  226068
i:  1897  index:  631066
i:  1904  index:  434440
i:  1905  index:  742684
i:  1904  index:  912706
i:  1899  index:  144503
i:  1900  index:  312550
i:  1901  index:  531102
i:  1904  index:  226069
i:  1898  index:  631067
i:  1905  index:  434450
i:  1905  index:  912707
i:  1906  index:  742685
i:  1900  index:  144504
i:  1901  index:  312560
i:  1902  index:  531103
i:  1905  index:  226070
i:  1899  index:  631068
i:  1906  index:  434460
i:  1906  index:  912708
i:  1901  index:  144505
i:  1907  index:  742686
i:  1902  index:  312570
i:  1903  index:  531104
i:  1900  index:  631069
i:  1906  index:  226080
i:  1907  index:  434470
i:  1907  index:  912709
i:  1902  index:  144506
i:  1908  index:  742687
i:  1904  index:  531105
i:  1901  index:  631070


i:  1943  index:  227383
i:  1939  index:  312673
i:  1941  index:  531530
i:  1939  index:  632401
i:  1939  index:  145205
i:  1944  index:  434700
i:  1943  index:  913460
i:  1944  index:  227384
i:  1945  index:  742846
i:  1940  index:  312674
i:  1942  index:  531540
i:  1940  index:  632402
i:  1940  index:  145206
i:  1945  index:  434810
i:  1944  index:  913471
i:  1945  index:  227385
i:  1941  index:  312675
i:  1946  index:  742847
i:  1943  index:  531550
i:  1941  index:  632403
i:  1941  index:  145207
i:  1946  index:  434821
i:  1945  index:  913472
i:  1946  index:  227386
i:  1942  index:  312676
i:  1944  index:  531560
i:  1947  index:  742848
i:  1942  index:  632404
i:  1942  index:  145208
i:  1947  index:  434822
i:  1946  index:  913473
i:  1947  index:  227387
i:  1943  index:  312677
i:  1945  index:  531570
i:  1943  index:  632405
i:  1943  index:  145209
i:  1948  index:  742849
i:  1948  index:  434823
i:  1947  index:  913474
i:  1948  index:  227388


i:  1984  index:  227698
i:  1981  index:  531807
i:  1980  index:  145773
i:  1985  index:  742899
i:  1981  index:  633041
i:  1985  index:  434927
i:  1980  index:  312812
i:  1985  index:  913803
i:  1985  index:  227699
i:  1982  index:  531808
i:  1981  index:  145774
i:  1986  index:  742900
i:  1982  index:  633042
i:  1986  index:  434928
i:  1986  index:  913804
i:  1986  index:  227700
i:  1981  index:  312813
i:  1983  index:  531809
i:  1982  index:  145775
i:  1987  index:  742911
i:  1983  index:  633043
i:  1987  index:  434929
i:  1987  index:  913805
i:  1987  index:  227801
i:  1984  index:  531901
i:  1982  index:  312814
i:  1983  index:  145776
i:  1988  index:  742912
i:  1984  index:  633044
i:  1988  index:  434930
i:  1988  index:  913806
i:  1988  index:  227802
i:  1985  index:  531902
i:  1984  index:  145777
i:  1989  index:  742913
i:  1983  index:  312815
i:  1985  index:  633045
i:  1989  index:  434941
i:  1989  index:  913807
i:  1989  index:  227803


i:  2021  index:  146206
i:  2020  index:  312935
i:  2022  index:  634114
i:  2022  index:  537009
i:  2026  index:  914507
i:  2026  index:  434982
i:  2025  index:  231203
i:  2027  index:  742953
i:  2022  index:  146207
i:  2021  index:  312936
i:  2023  index:  634115
i:  2023  index:  538000
i:  2027  index:  914508
i:  2027  index:  434983
i:  2026  index:  231204
i:  2028  index:  742954
i:  2023  index:  146208
i:  2022  index:  312937
i:  2024  index:  541000
i:  2024  index:  634116
i:  2028  index:  914509
i:  2028  index:  434984
i:  2027  index:  231205
i:  2029  index:  742955
i:  2024  index:  146209
i:  2023  index:  312938
i:  2025  index:  542000
i:  2029  index:  914600
i:  2029  index:  434985
i:  2028  index:  231206
i:  2025  index:  634117
i:  2030  index:  742956
i:  2025  index:  146300
i:  2024  index:  312939
i:  2026  index:  543000
i:  2030  index:  914700
i:  2030  index:  434986
i:  2029  index:  231207
i:  2026  index:  634118
i:  2031  index:  742957


[INFO/ForkPoolWorker-3] process shutting down
[INFO/ForkPoolWorker-2] process shutting down
[INFO/ForkPoolWorker-4] process shutting down
[INFO/ForkPoolWorker-7] process shutting down
[INFO/ForkPoolWorker-6] process shutting down
[INFO/ForkPoolWorker-1] process shutting down
[INFO/ForkPoolWorker-3] process exiting with exitcode 0
[INFO/ForkPoolWorker-5] process shutting down
[INFO/ForkPoolWorker-8] process shutting down
[INFO/ForkPoolWorker-2] process exiting with exitcode 0
[INFO/ForkPoolWorker-4] process exiting with exitcode 0
[INFO/ForkPoolWorker-1] process exiting with exitcode 0
[INFO/ForkPoolWorker-5] process exiting with exitcode 0
[INFO/ForkPoolWorker-6] process exiting with exitcode 0
[INFO/ForkPoolWorker-8] process exiting with exitcode 0
[INFO/ForkPoolWorker-7] process exiting with exitcode 0


In [25]:
df_complete = pd.concat(pool.map(addUpstream2, indices_split))

In [26]:
df_complete = pd.concat(pool.map(addDownstream2, indices_split))

In [27]:
df_complete = pd.concat(pool.map(addBasin2, indices_split))

In [28]:
pool.close()

In [29]:
df_complete.to_pickle(os.path.join(EC2_OUTPUT_PATH,OUTPUT_FILENAME+".pkl"))

In [30]:
df_complete.to_csv(os.path.join(EC2_OUTPUT_PATH,OUTPUT_FILENAME+".csv"))

In [31]:
!aws s3 cp {EC2_OUTPUT_PATH} {S3_OUTPUT_PATH} --recursive

upload: ../../../../data/Y2017M09D15_RH_Add_Basin_Data_V01/output/Y2017M09D15_RH_Add_Basin_Data_V01.pkl to s3://wri-projects/Aqueduct30/processData/Y2017M09D15_RH_Add_Basin_Data_V01/output/Y2017M09D15_RH_Add_Basin_Data_V01.pkl
upload: ../../../../data/Y2017M09D15_RH_Add_Basin_Data_V01/output/Y2017M09D15_RH_Add_Basin_Data_V01.csv to s3://wri-projects/Aqueduct30/processData/Y2017M09D15_RH_Add_Basin_Data_V01/output/Y2017M09D15_RH_Add_Basin_Data_V01.csv


In [32]:
df_complete.head()

,HYBAS_ID2,Unnamed: 0,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,...,basin_total_volume_runoff_monthY2014M02,basin_total_volume_PLivWN_monthY2014M10,basin_total_volume_PDomWN_yearY2014M12,basin_total_volume_runoff_monthY2014M10,basin_total_volume_PIrrWW_monthY2014M08,basin_total_volume_IrrLinearWW_monthY2014M04,basin_total_volume_PLivWW_yearY2014M12,basin_total_volume_PIrrWN_monthY2014M07,basin_total_volume_IrrLinearWN_monthY2014M05,errorCode
PFAF_ID,,,,,,,,,,,,,,,,,,,,,
111011,1060000010,6675,1060000010,0,1060000010,1060000010,0.0,0.0,1890.8,1890.8,...,145276.307683,248.908290,8.642677e+06,37519.523523,447162.941202,294146.068317,2897.213351,134465.393295,89361.642352,0.0
111012,1060000100,6676,1060000100,0,1060000100,1060000100,0.0,0.0,2925.9,2925.9,...,60792.150023,1873.557120,1.561193e+05,16735.674093,0.000000,0.000000,21834.864422,0.000000,0.000000,0.0
111013,1060000110,6677,1060000110,0,1060000110,1060000110,0.0,0.0,893.5,893.5,...,18066.833124,44.161986,1.390749e+05,4394.811805,0.000000,0.000000,514.531435,0.000000,0.000000,0.0
111014,1060000150,6678,1060000150,0,1060000150,1060000150,0.0,0.0,4217.3,4217.4,...,0.000000,858.784192,1.295654e+05,0.000000,0.000000,0.000000,10041.114809,0.000000,0.000000,0.0
111015,1060000160,6679,1060000160,0,1060000160,1060000160,0.0,0.0,16638.1,16638.1,...,-0.145698,430.713409,3.560829e+06,342.706117,0.000000,0.000000,5038.263451,0.000000,0.000000,0.0
